In [16]:
import os
import json
from dotenv import load_dotenv
load_dotenv('/Users/joe.constantino/Desktop/playground/tableau_langchain/.env')

#base langchain library imports
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent, AgentExecutor

#tableau_langchain imports
from utilities.tableau.readMetadata import read
from utilities.tableau.setPrompt import instantiate_prompt
from tools.tableau.getDataTool import get_data

ModuleNotFoundError: No module named 'tableau_langchain.utilities'

In [13]:
#read in the environment varibales to query a Tableau Published Data Source from a target Tableau Cloud Site or Server Instance
datasource = os.getenv('DATASOURCE_LUID')
read_metadata_url = os.getenv('READ_METADATA')
query_datasource = os.getenv('QUERY_DATASOURCE')
token = os.getenv('AUTH_TOKEN')

In [14]:
## Use the read module to pull in the necessary metadata from your target Tableau Published Data Source. This will module also augment the
## base metadata provided by the read_metadata endpoint by removing unneccesary fields and adding sample field values to each string field.
metadata = read(read_url=read_metadata_url, datasource_luid=datasource, auth_secret=token)

# use the instaniate_prompt module to add the data model metadata to the nlq_to_vds prompt
instantiate_prompt(metadata = metadata)

reading in your field metadata...
looking up field values...
prompt is ready!


{'instructions': "\nYou are an expert at writing API request bodies for Tableau’s HeadlessBI API.\nThe HBI query is a JSON object that contains 2 fundamental components.\n    1. fields [required] - an array of fields that define the desired output of the query\n    2. filters [optional] - an array of filters to apply to the query. They can include fields that are not in the fields array.\nYour task is to retrieve data relevant to a user’s natural language query.\n\nQuery as much data as might be useful; it's ok if you pull in superfluous fields,\nYou will be successful if you bring back all the data that could help to answer the question, even if additional transformation and actions are needed.\n\nYou can find the fieldCaptions by checking the data_model field.\nKeep your output very structured. Use the following structure:\nReasoning:\n\nJSON_payload:\nMake sure you use this structure so that it's simple to parse the output.\nReturn query results verbatim so the pandas agent can anal

In [15]:
# Initialize an LLM and an agent with the get_data tool to answer natural language questions with Tableau Published Datasources
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
tools = [get_data]

tableauHeadlessAgent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS, # Use OpenAI's function calling
    verbose = True
)

# Run the agent
response = tableauHeadlessAgent.invoke("total sales per state")
print(response)

/var/folders/3t/f2z2c6_n3951jmt86v5rvgp80000gp/T/ipykernel_70668/1891676.py:5: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  tableauHeadlessAgent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `get_data` with `{'query': 'total sales per state'}`


[{'State/Province': 'Florida', 'SUM(Sales)': 89473.71}, {'State/Province': 'Oregon', 'SUM(Sales)': 17431.15}, {'State/Province': 'New Hampshire', 'SUM(Sales)': 7292.52}, {'State/Province': 'New Brunswick', 'SUM(Sales)': 225.7}, {'State/Province': 'Minnesota', 'SUM(Sales)': 29863.15}, {'State/Province': 'North Dakota', 'SUM(Sales)': 919.91}, {'State/Province': 'Newfoundland and Labrador', 'SUM(Sales)': 515.2}, {'State/Province': 'Arizona', 'SUM(Sales)': 35282.0}, {'State/Province': 'Iowa', 'SUM(Sales)': 4579.76}, {'State/Province': 'Rhode Island', 'SUM(Sales)': 22627.96}, {'State/Province': 'Arkansas', 'SUM(Sales)': 11678.13}, {'State/Province': 'Ontario', 'SUM(Sales)': 7009.01}, {'State/Province': 'Manitoba', 'SUM(Sales)': 1930.78}, {'State/Province': 'Texas', 'SUM(Sales)': 170188.05}, {'State/Province': 'Mississippi', 'SUM(Sales)': 10771.34}, {'State/Province': 'Nevada', 'SUM(Sale